In [ ]:
import pandas as pd

#two input files
ratesFile = 'FA19_UK_LM_status_rates.csv'
levelsFile = 'FA19_UK_LM_status_levels.csv'

originalFile = 'v4-lms.csv'

dfRates = pd.read_csv(ratesFile, dtype = str)
dfLevels = pd.read_csv(levelsFile, dtype = str)

#renaming the columns we are interested in
renameCols = {
            'obs':'V4_0',
            'Time dim it id':'time',
            'dim1itid':'economicstatus',
            'dim2itid':'age',
            'dim3itid':'sex',
            'dim4itid':'seasonaladjustment'
            }

dfRates = dfRates.rename(columns = renameCols)
dfLevels = dfLevels.rename(columns = renameCols)

#adding in codelist columns
dfRates['mmm-mmm-yyyy'] = dfRates['time']
dfRates['uk-only'] = 'K02000001'
dfRates['geography'] = 'United Kingdom'
dfRates['unit-of-measure'] = 'rates'
dfRates['unitofmeasure'] = 'Rates'
dfRates['lms-economic-status'] = dfRates['economicstatus']
dfRates['lms-age-bracket'] = dfRates['age']
dfRates['seasonal-adjustment'] = dfRates['seasonaladjustment']

dfLevels['mmm-mmm-yyyy'] = dfLevels['time']
dfLevels['uk-only'] = 'K02000001'
dfLevels['geography'] = 'United Kingdom'
dfLevels['unit-of-measure'] = 'levels'
dfLevels['unitofmeasure'] = 'Levels'
dfLevels['lms-economic-status'] = dfLevels['economicstatus']
dfLevels['lms-age-bracket'] = dfLevels['age']
dfLevels['seasonal-adjustment'] = dfLevels['seasonaladjustment']

dfRates = dfRates[[
        'V4_0', 'mmm-mmm-yyyy', 'time', 'uk-only', 'geography',
        'unit-of-measure', 'unitofmeasure', 'lms-economic-status','economicstatus', 
        'lms-age-bracket', 'age', 'sex', 'seasonal-adjustment', 'seasonaladjustment'
        ]]

dfLevels = dfLevels[[
        'V4_0', 'mmm-mmm-yyyy', 'time', 'uk-only', 'geography',
        'unit-of-measure', 'unitofmeasure', 'lms-economic-status','economicstatus', 
        'lms-age-bracket', 'age', 'sex', 'seasonal-adjustment', 'seasonaladjustment'
        ]]

'''Functions'''

def SeasonalValues(value):
    if value.startswith('Non'):
        return 'Non Seasonal Adjustment'
    else:
        return 'Seasonal Adjustment'
    
def SeasonalValueChanges(value):
    lookup = {
            'Non Seasonal Adjustment':'Not Seasonally Adjusted',
            'Seasonal Adjustment':'Seasonally Adjusted'
            }
    return lookup[value]

'''Post Processing'''

dfRates['mmm-mmm-yyyy'] = dfRates['mmm-mmm-yyyy'].apply(lambda x:x.replace(' ','-').lower())
dfLevels['mmm-mmm-yyyy'] = dfLevels['mmm-mmm-yyyy'].apply(lambda x:x.replace(' ','-').lower())

dfRates['lms-economic-status'] = dfRates['lms-economic-status'].apply(lambda x:x.replace(' ','-')
                                                                            .lower())
dfLevels['lms-economic-status'] = dfLevels['lms-economic-status'].apply(lambda x:x.replace(' ','-')
                                                                            .lower())

dfRates['adult-sex'] = dfRates['sex'].apply(lambda x:x.lower())
dfLevels['adult-sex'] = dfLevels['sex'].apply(lambda x:x.lower())

dfRates['seasonaladjustment'] = dfRates['seasonaladjustment'].apply(SeasonalValues)
dfRates['seasonal-adjustment'] = dfRates['seasonaladjustment'].apply(lambda x:x.replace(' ','-')
                                                                            .lower())
dfRates['seasonaladjustment'] = dfRates['seasonaladjustment'].apply(SeasonalValueChanges)

dfLevels['seasonaladjustment'] = dfLevels['seasonaladjustment'].apply(SeasonalValues)
dfLevels['seasonal-adjustment'] = dfLevels['seasonaladjustment'].apply(lambda x:x.replace(' ','-')
                                                                            .lower())
dfLevels['seasonaladjustment'] = dfLevels['seasonaladjustment'].apply(SeasonalValueChanges)

#combining the two
df = pd.concat([dfRates, dfLevels])

#moving data markings to a separate column
df.loc[df['V4_0'] == '*', 'Data_Marking'] = '*'
df = df.rename(columns = {'V4_0':'V4_1'})
df = df[[
        'V4_1', 'Data_Marking', 'mmm-mmm-yyyy', 'time', 'uk-only', 'geography',
        'unit-of-measure', 'unitofmeasure', 'lms-economic-status', 'economicstatus', 
        'lms-age-bracket', 'age', 'adult-sex', 'sex', 'seasonal-adjustment', 'seasonaladjustment'
        ]]

originalDF = pd.read_csv(originalFile, dtype = str)
originalDF = pd.concat([originalDF, df]).drop_duplicates()

originalDF.to_csv(originalFile, index = False)